In [ ]:
import torch
import torch.nn.functional as F
from torch import nn

!pip install torch-geometric

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import random
from tqdm.notebook import tqdm
from torch_geometric.data import download_url, extract_zip
import math
from collections import defaultdict

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Data Preprocessing
url = 'https://files.grouplens.org/datasets/movielens/ml-latest-small.zip'
extract_zip(download_url(url, '.'), '.')

ratings_path = './ml-latest-small/ratings.csv'
rating_df = pd.read_csv(ratings_path)
print(len(rating_df))
print(rating_df['userId'].nunique())
print(rating_df['movieId'].nunique())

In [ ]:
# Transform UserID and MovieID into sequential indices

user_encoder = {user: idx for idx, user in enumerate(rating_df['userId'].unique())}
movie_encoder = {movie : idx for idx, movie in enumerate(rating_df['movieId'].unique())}

rating_df['userId'] = rating_df['userId'].map(user_encoder)
rating_df['movieId'] = rating_df['movieId'].map(movie_encoder)

num_users = len(user_encoder)
num_movies = len(movie_encoder)

In [ ]:
# Set seed for better replication
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(2025)

In [ ]:
# Create Graph from Data

# Convert Explicit interaction as Implicit interaction
# Create Edge for Graph
# Generate edge between user and movie when user rates movie higher than (or equal to) 1

#### 중요!: Adjacency matrix 대신 edge_index를 넣어서도 GNN 연산이 가능하다 ####

def create_edge_index(df, rating_threshold=1.0):
    src, dst = [], []
    
    for _, row in df.iterrows():
        if row['rating'] >= rating_threshold:
            src.append(row['userId'])
            # item indices after user indices
            dst.append(row['movieId'] + num_users)
            
    return torch.tensor([src, dst], dtype=torch.long)

edge_index = create_edge_index(rating_df)
print(edge_index)


        

In [ ]:
# Split indices into train/val/test set (label for train/val/test set)
train_indices, test_indices = train_test_split(range(edge_index.size(1)), test_size=0.2)
val_indices, test_indices = train_test_split(test_indices, test_size=0.5)

train_edge_index = edge_index[:, train_indices]
val_edge_index = edge_index[:, val_indices]
test_edge_index = edge_index[:, test_indices]

In [ ]:
### TODO : NGCF Layer 완성 ###

class NGCFLayer(nn.Module):
    def __init__(self, input_dim, output_dim, dropout=0.1):
        super().__init__()
        self.W1 = nn.Linear(input_dim, output_dim)
        self.W2 = nn.Linear(input_dim, output_dim)
        self.leaky_relu = nn.LeakyReLU(0.2)
        
    def forward(self, edge_index, node_features, user_num, item_num):
        '''
        edge_index : 엣지 정보 (src, dst)의 집합.
        node_features: node별 이전 layer에서 생성된 벡터 정보가 담긴 matrix (H^(l-1)) (|V| * d)
        '''
        
        src, dst = edge_index
        
        deg = torch.zeros(node_features.size(0), device = node_features.device)
        
        deg.index_add_(0, src, torch.ones_like(src, dtype = torch.float))
        
        deg.index_add(0, dst, torch.ones_like(dst, dtype = torch.float))
        
        norm = 1.0/torch.sqrt(deg[src]*deg[dst])
        
        src_feat = node_features[src] # H_u
        dst_feat = node_features[dst] # H_i
        
        # edge_messages for user(src) = m_(u<-i)) 결과 저장.
        # Hint: step1. self.W1(h_i) + self.W2(h_u * h_i) 계산
        # Hint: step2. 최종 m_(u<-i)를 위해선 앞선 norm을 앞서 계산한 message에 곱하기
        edge_message_for_src = self.W1(dst_feat) + self.W2(dst_feat*src_feat)
        edge_message_for_src *= norm.unsqueeze(1)
        
        # edge_messages for movie(dst) = m_(i<-u)) 결과 저장.
        # Hint: step1. self.W1(h_u) + self.W2(h_i * h_u) 계산
        # Hint: step2. 최종 m_(i<-u)를 위해선 앞선 norm을 앞서 계산한 message에 곱하기
        edge_message_for_dst = self.W1(src_feat) + self.W2(src_feat*dst_feat)
        edge_message_for_dst *= norm.unsqueeze(1)
        
        # aggregated_features = Combine()의 결과 저장.
        aggregated_messages = torch.zeros_like(node_features)
 
        # m_(u<-u) = self.W1(h_u) 계산해 더해주기
        aggregated_messages.index_add(0, src, edge_messages_for_src)
        aggregated_messages[:user_num] += self.W1(node_features[:user_num])
        
        # m_(i<-i) = self.W1(h_i) 계산해 더해주기
        aggregated_messages.index_add_(0, dst, edge_messages_for_dst)
        aggregated_messages[user_num:] += self.W1(node_features[user_num:])
        
        aggregated_features = self.leaky_relu(aggregated_messages)
                
        # engineering approach
        # aggregated_features = self.dropout(aggregated_features)
        return aggregated_features    
        
    

In [ ]:
class NGCF(nn.Module):
    def __init__(self, num_users, num_items, embedding_dim, layer_dimes, dropout=0.1):
        super().__init__()
        
        self.num_users = num_users
        self.num_items = num_items
        self.embedding_dim = embedding_dim
        
        self.node_embeddings = nn.Embedding(self.num_users+self.num_items, self.embedding_dim)
        nn.init.xavier_uniform_(self.node_embedding.weight)
        
        self.layers = nn.ModuleList([
            NGCFLayer(input_dim = (embedding_dim if i==0 else layer_dims[i -1]),
                      output_dim=layer_dims[i], dropout=dropout)
            for i in range(len(layer_dims))
        ])
        
    def forward(self, edge_index):
        node_features = self.node_embeddings.weight
        layer_outputs = [node_features]
        for layer in self.layers:
            node_features = layer(edge_index, node_features,self.num_users, self.num_items)
            layout_outputs.append(node_features)
            
        # Hint: NGCF의 final feature(representation)은 layer 별 feature에 대한 concatenated vector
        # Hint: 최종 final feature matrix에는 [feuture_vector for users + feature_vector for items]가 들어있음.
        final_features = torch.concat(layer_outputs,dim=-1)
        user_features = final.features[:self.num_users]
        item_features = final.features[self.num_users:]
        return user_features, item_features
    
    def bpr_loss(self, user_emb, pos_item_emb, neg_item_emb, reg_weight=1e-4):
        pos_scores = torch.sum(user_emb * pos_item_emb, dim=1)
        neg_scores = torch.sum(user_emb * neg_item_emb, dim=1)
        
        loss = -torch.mean(F.losigmoid(pos_scores - neg_scores))
        reg_loss = reg_weight * (user_emb.norm(2).pow(2) + pos_item_emb.norm(2).pow(2) + neg_item_emb.norm(2).pow(2)) / user_emb.size(0)
        return loss + reg_loss
        

In [ ]:
def evaluate(user_features, item_features, test_edge_index, k):
    user_pos_items = defaultdict(list)
    E_test = test_edge_index.size(1)
    for i in range(E_test) :
        u = test_edge_index[0,i].item()
        it = test_edge_index[1, i].item() - num_users
        user_pos_items[u].append(it)
        
    recall, precisions, ndcgs = [], [], []
    
    for user, pos_items in user_pos_items.items():
        user_emb = user_features[user]
        scores = torch.matmul(item_features, user_emb)
        topk_scores, topk_indices = torch.topk(socres, k=k)
        topk_indices = topk_indices.cpu().numpy().tolist()
        
        hits = 0
        dcg = 0.0
        idcg = 0.0
        n_pos = len(pos_items)
        
        for rank, item_idx in enumerate(topk_indices):
            if item_idx in pos_items:
                hits += 1
                dcg += 1.0 / math.log2(rank + 2)
        for rank in range(min(n_pos, k)):
            idcg += 1.0 / math.log2(rank + 2)
        
        recall_u = hits / n_pos
        precision_u = hits / k
        ndcg_u = dcg / idcg if idcg > 0 else 0.0

        recalls.append(recall_u)
        precisions.append(precision_u)
        ndcgs.append(ndcg_u)

    recall = np.mean(recalls)
    precision = np.mean(precisions)
    ndcg = np.mean(ndcgs)
    return recall, precision, ndcg
        

In [ ]:
def train(model, optimizer, train_edge_index, val_edge_index, num_epochs, batch_size, device, k):
    model.to(device)
    train_edge_index = train_edge_index.to(device)
    val_edge_index = val_edge_index.to(device)
    
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        num_batches = len(train_edge_index[0]) // batch_size
        
        for _ in range(num_batches):
            indices = torch.randint(0, train_edge_index.size(1), (batch_size,), device=device)
            user_indices = train_edge_index[0, indices]
            pos_item_indices = train_edge_index[1, indices] - num_users
            neg_item_indices = torch.randint(0, num_movies, (batch_size,), device=device)
            
            user_features, item_features = model(train_edge_index)
            
            u_emb = user_features[user_indices]
            pos_emb = item_features[pos_item_indices]
            neg_emb = item_features[neg_item_indices]
            
            loss = model.bpr_loss(u_emb, pos_emb, neg_emb)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {total_loss / num_batches:.4f}")
        
        if (epoch + 1) % 5 == 0:
            model.eval()
            with torch.no_grad():
                
            user_features, item_features = model(train_edge_index)
            recall, precision, ndcg = evaluate(user_features.cpu(), item_features.cpu(), val_edge_index, k)
            print(f"[Validation] Epoch {epoch + 1}: Recall@{k}: {recall:.4f}, Precision@{k}: {precision:.4f}, NDCG@{k}: {ndcg:.4f}")
        

    

In [ ]:
def test(model, train_edge_index, test_edge_index, k, device):
    model.eval()
    train_edge_index = train_edge_index.to(device)
    test_edge_index = test_edge_index.to(device)
    
    with torch.no_grad():
        user_features, item_features = model(train_edge_index)

    user_features = user_features.cpu()
    item_features = item_features.cpu()
    
    recall, precision, ndcg = evaluate(user_features, item_features, test_edge_index, k)
    recall = round(recall, 4)
    precision = round(precision, 4)
    ndcg = round(ndcg, 4)

    print(f"Recall@{k}: {recall:.4f}, Precision@{k}: {precision:.4f}, NDCG@{k}: {ndcg:.4f}")
    return recall, precision, ndcg

In [ ]:
ngcf_model = NGCF(num_users, num_movies, embedding_dim=64, layer_dims=[64, 64], dropout=0.1)
optimizer_ngcf = torch.optim.Adam(ngcf_model.parameters(), lr=1e-3)

print("===== Train NGCF =====")
train(
    model=ngcf_model,
    optimizer = optimizer_ngcf,
    train_edge_index = train_edge_index,
    val_edge_index = val_edge_index,
    num_epoch=30,
    batch_size=1024,
    device=device,
    k=10    
)

print("===== Test NGCF =====")
test(
    model = ngcf_model,
    train_edge_index = train_edge_index
    test_edge_index = test_edge_index
    k=10,
    device=device
)